In [1]:
import pandas as pd
from scipy.stats import norm
import numpy as np
from datetime import datetime, timedelta


**Задание 3**

Сколько пользователей нужно взять в экспериментальную группу, чтобы отлавливать эффект
в 20 копеек? Известно, что для эффекта в 10 копеек нужно 200000 пользователей. Вероятности ошибок
и стандартные отклонения фиксированы.

***Решение***

Если вероятность ошибок (альфа и бета) и стандартное отклонение фиксированы, то объем выборки для выявления эффекта прямо пропорционален квадрату величины эффекта. Это связано с тем, что дисперсия уменьшается пропорционально 1/n, а требуемая точность зависит от размера эффекта


In [2]:
n1 = 200000
d2 = 0.2
d1 = 0.1

n1/n2 = (d2/d1)**2

n2 = n1*d1**2/d2**2

In [3]:
n2 = n1*d1**2/d2**2
n2

50000.0

Ответ: Для отлавливания эффекта в 20 копеек требуется 50,000 пользователей.

**Задание 4**


Мы хотим проверить гипотезу об увеличении средней выручки с покупателя. По историческим данным мы вычислили, что оценка среднего значения метрики равна 1230 рублей, а оценка стандартного отклонения равна 200 рублей. Размер групп в эксперименте фиксирован и равен 1000. Эффекты какого размера можем отловить на уровне значимости 0.05 и допустимой вероятности ошибки II рода 0.1 ?

Решение

In [4]:

# Данные
mu = 1230  # среднее значение
sigma = 200  # стандартное отклонение
n = 500 # размер группы
alpha = 0.05  # уровень значимости
beta = 0.1  # допустимая вероятность ошибки II рода

# Критические значения для Z-распределения
z_alpha = norm.ppf(1 - alpha / 2)  # для двустороннего теста
z_beta = norm.ppf(1 - beta)

# Минимально обнаруживаемый эффект (MDE)
mde = (z_alpha + z_beta) * (sigma / np.sqrt(n))

mde

28.992996480447648

n = 500, т.к. группы 2, в сумме 1000, значит каждая 500

In [5]:
from scipy.stats import t

# Используем t-распределение вместо нормального
df = n - 1  # степени свободы

# Критические значения для t-распределения
t_alpha = t.ppf(1 - alpha / 2, df)  # для двустороннего теста
t_beta = norm.ppf(1 - beta)  # для мощности все еще используем нормальное распределение

# Перерасчет MDE для t-теста
mde_t_test = (t_alpha + t_beta) * (sigma / np.sqrt(n))

mde_t_test

29.035619571476744

Ответ: 30 руб 

---------------

## Семинар

**Способы вычисления смещённой и несмещённой оценки дисперсии**

In [6]:
values = np.arange(5)

# смещённые оценки дисперсии равны 2.0
var_1 = ((values - values.mean()) ** 2).mean()
var_2 = values.var()
var_3 = pd.Series(values).var(ddof=0)

# несмещённые оценки дисперсии равны 2.5
var_1 = ((values - values.mean()) ** 2).sum() / (len(values) - 1)
var_2 = values.var(ddof=1)
var_3 = pd.Series(values).var()

**Задача 1. Оценить необходимый размер групп**

Допустим, мы хотим провести эксперимент, в который попадают клиенты, совершившие покупку во время эксперимента.

Метрика — средняя выручка с пользователя за время эксперимента;

Продолжительность — одна неделя;

Уровень значимости — 0.05;

Допустимая вероятность ошибки II рода — 0.1;

Ожидаемый эффект — 20 рублей.

Оцените необходимый размер групп по данным о покупках за неделю с 21 по 28 февраля. Обратим внимание, что в выборку попадают события из полуинтервала `[datetime(2022, 2, 21), datetime(2022, 2, 28))`

In [7]:
df_sales = pd.read_csv('./2022-04-01T12_df_sales.csv')
df_sales.tail(3)

,sale_id,date,count_pizza,count_drink,price,user_id
203844,1203845,2022-04-01 11:59:43,1,0,600,7cdcc7
203845,1203846,2022-04-01 11:59:45,1,1,630,47b825
203846,1203847,2022-04-01 11:59:51,1,0,780,cdaabb


In [8]:
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_sales_per = df_sales[(df_sales['date'] >= datetime(2022, 2, 21))\
                        &(df_sales['date'] < datetime(2022, 2, 28))]\
                            .copy(deep=True)
df_sales_per.tail(3)

,sale_id,date,count_pizza,count_drink,price,user_id
88551,1088552,2022-02-27 21:59:36,2,0,1500,3d5e65
88552,1088553,2022-02-27 21:59:41,1,0,660,63bff8
88553,1088554,2022-02-27 21:59:54,4,0,3060,601282


In [9]:
df_user_revenue = df_sales_per.groupby('user_id')['price'].sum().reset_index()
df_user_revenue.tail()

,user_id,price
24835,ffec21,2130
24836,fff223,540
24837,fff4db,840
24838,fff718,780
24839,fff7ff,780


In [10]:
def get_sample_size_abs(epsilon, std, alpha=0.05, beta=0.1):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    z_scores_sum_squared = (t_alpha + t_beta) ** 2
    sample_size = int(
        np.ceil(
            z_scores_sum_squared * (2 * std ** 2) / (epsilon ** 2)
        )
    )
    return sample_size

def get_sample_size_arb(mu, std, eff=1.01, alpha=0.05, beta=0.1):
    epsilon = (eff - 1) * mu

    return get_sample_size_abs(epsilon, std=std, alpha=alpha, beta=beta)

In [11]:
mu = df_user_revenue['price'].mean()
std = df_user_revenue['price'].std()
eff = 1+(20/mu)
alpha = 0.05
beta = 0.1


sample_size = get_sample_size_arb(mu, std, eff, alpha=0.05, beta=0.1)
print(f'sample_size rounded = {round(sample_size, -1)}\n')

sample_size rounded = 34570



In [12]:
sample_size = get_sample_size_abs(20, std, alpha=0.05, beta=0.1)
print(f'sample_size rounded = {round(sample_size, -1)}\n')

sample_size rounded = 34570



Ответ: 34570

**Задача 2. MDE**
  

В прошлом задании получилось, что необходимый размер групп больше имеющихся данных за одну неделю. Какой минимальный эффект мы можем отловить с теми же вероятностями ошибок на данных
с 21 по 28 февраля?

Ответ округлите до целого значения.

In [13]:
def get_minimal_determinable_effect(std, sample_size, alpha=0.05, beta=0.2):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    disp_sum_sqrt = (2 * (std ** 2)) ** 0.5
    mde = (t_alpha + t_beta) * disp_sum_sqrt / np.sqrt(sample_size)
    return mde

In [14]:
mde = get_minimal_determinable_effect(std, df_user_revenue['user_id'].count()/2, alpha=0.05, beta=0.1)

print(f'mde = {mde}\n')

mde = 33.36719696756425



**Задача 3. Функция оценки размера выборки**

Реализуйте функцию estimate_sample_size.

Обратите внимание:

1. Размер эффекта задаётся в процентах;

2. Для вычисления стандартного отклонения используйте функцию np.std с параметрами по умолчанию.

3. Не используйте агрегацию внутри функции.

4. Стандартное отклонение и значение среднего необходимо посчитать по полученному столбцу с метрикой.



***Решение***

In [15]:
import numpy as np
import pandas as pd
from scipy import stats


def estimate_sample_size(metrics, effect, alpha, beta):
    """Оцениваем необходимый размер выборки для проверки гипотезы о равенстве средних.
    
    Для метрик, у которых для одного пользователя одно значение просто вычислите
    размер групп по формуле.
    Для метрик, у которых для одного пользователя несколько значений (например,
    response_time), вычислите необходимый объём данных и разделите его на среднее
    количество значений на одного пользователsя.
    Пример, если в таблице metrics 1000 наблюдений и 100 уникальных пользователей,
    и для эксперимента нужно 302 наблюдения, то размер групп будет 31, тк в среднем на
    одного пользователя 10 наблюдений, то получится порядка 310 наблюдений в группе.

    :param metrics (pd.DataFrame): таблица со значениями метрик,
        содержит столбцы ['user_id', 'metric'].
    :param effect (float): размер эффекта в процентах.
        Пример, effect=3 означает, что ожидаем увеличение среднего на 3%.
    :param alpha (float): уровень значимости.
    :param beta (float): допустимая вероятность ошибки II рода.
    :return (int): минимально необходимый размер групп (количество пользователей)
    """
    nunique_users = metrics['user_id'].nunique()
    cnt_val = len(metrics)

    mu = metrics['metric'].mean()
    std = np.std(metrics['metric'])

    effect = effect/100 * mu

    t_alpha = norm.ppf(1-alpha/2)
    t_beta = norm.ppf(1-beta)

    z_scores_sum_squared = (t_alpha + t_beta)**2
    sample_size = int(np.ceil(z_scores_sum_squared*(2*std**2)/(effect**2)))

    if nunique_users==cnt_val:
        sample_size_final = sample_size
        return sample_size_final
    else:
        avg_cnt_per_user = metrics.groupby(['user_id']).agg(cnt = ('metric', 'count')).reset_index()['cnt'].mean()
        sample_size_final = int(np.ceil(sample_size/avg_cnt_per_user))
        return sample_size_final

***Пример***

In [16]:
effect, alpha, beta = 3, 0.05, 0.1

metrics = pd.DataFrame({
    'user_id': np.arange(100),
    'metric': np.linspace(500, 1490, 100)
})

sample_size = estimate_sample_size(metrics, effect, alpha, beta)
print(sample_size)

1966


In [17]:
metrics = pd.DataFrame({
    'user_id': np.arange(100) % 30,
    'metric': np.linspace(500, 1490, 100)
})
effect, alpha, beta = 3, 0.05, 0.1
sample_size = estimate_sample_size(metrics, effect, alpha, beta)
print(sample_size)

590
